In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from animate import *
import sys
sys.path.append('../../src/')
from helper_functions import *
plt.style.use('dynamics_site')

## Define Integration Loop

In [2]:
def integrate_rk45_dp(system,dt,steps,thresh=1e-9):
    G, mvec, rvec, vvec = load_initials(system)
    Rs = [rvec]
    Vs = [vvec]
    dts = [dt]
    for i in range(steps-1):
        k1_vi = dt*get_acclerations(rvec,mvec)
        k1_ri = dt*vvec
        
        k2_vi = dt*get_acclerations(rvec +(1/5)*k1_ri,mvec)
        k2_ri = dt*(vvec                 +(1/5)*k1_vi)

        k3_vi = dt*get_acclerations(rvec +(3/40)*k1_ri + (9/40)*k2_ri,mvec)
        k3_ri = dt*(vvec                 +(3/40)*k1_vi + (9/40)*k2_vi)

        k4_vi = dt*get_acclerations(rvec +(44/45)*k1_ri - (56/15)*k2_ri + (32/9)*k3_ri,mvec)
        k4_ri = dt*(vvec                 +(44/45)*k1_vi - (56/15)*k2_vi + (32/9)*k3_vi)

        k5_vi = dt*get_acclerations(rvec +(19372/6561)*k1_ri - (25360/2187)*k2_ri + (64448/6561)*k3_ri - (212/729)*k4_ri,mvec)
        k5_ri = dt*(vvec                 +(19372/6561)*k1_vi - (25360/2187)*k2_vi + (64448/6561)*k3_vi - (212/729)*k4_vi)
                                                                            #several sources incorrectly use -(46732/5247) here!!
        k6_vi = dt*get_acclerations(rvec +(9017/3168)*k1_ri - (355/33)*k2_ri + (46732/5247)*k3_ri + (49/176)*k4_ri - (5103/18656)*k5_ri,mvec)
        k6_ri = dt*(vvec                 +(9017/3168)*k1_vi - (355/33)*k2_vi + (46732/5247)*k3_vi + (49/176)*k4_vi - (5103/18656)*k5_vi )

        k7_vi = dt*get_acclerations(rvec +(35/384)*k1_ri + (500/1113)*k3_ri + (125/192)*k4_ri - (2187/6784)*k5_ri + (11/84)*k6_ri,mvec)
        k7_ri = dt*(vvec                 +(35/384)*k1_vi + (500/1113)*k3_vi + (125/192)*k4_vi - (2187/6784)*k5_vi + (11/84)*k6_vi)

        new_vvec_order4 = vvec + (35/384)*k1_vi + (500/1113)*k3_vi + (125/192)*k4_vi - (2187/6784)*k5_vi + (11/84)*k6_vi
        new_rvec_order4 = rvec + (35/384)*k1_ri + (500/1113)*k3_ri + (125/192)*k4_ri - (2187/6784)*k5_ri + (11/84)*k6_ri
        new_vvec_order5 = vvec + (5179/57600)*k1_vi + (7571/16695)*k3_vi + (393/640)*k4_vi - (92097/339200)*k5_vi + (187/2100)*k6_vi + (1/40)*k7_vi
        new_rvec_order5 = rvec + (5179/57600)*k1_ri + (7571/16695)*k3_ri + (393/640)*k4_ri - (92097/339200)*k5_ri + (187/2100)*k6_ri + (1/40)*k7_ri

        maxerr = np.max([new_vvec_order5-new_vvec_order4,new_rvec_order5-new_rvec_order4])
        dt_new = (thresh/maxerr)**(1/5) * dt
        
        Rs.append(new_rvec_order4)
        Vs.append(new_vvec_order4)
        dts.append(dt_new)
        rvec = new_rvec_order4
        vvec = new_vvec_order4
        dt = dt_new

    Rs = np.array(Rs)
    Vs = np.array(Vs)
    PE = get_PE(Rs,mvec)
    KE = get_KE(Vs,mvec)
    
    Rs = np.array([Rs[j].T for j in range(len(Rs))])
    Vs = np.array([Vs[j].T for j in range(len(Rs))])
    return Rs,Vs,PE+KE,dts


## Preform Integration

In [3]:
dt = 0.01
steps = 20000
Rs,Vs,Es,Dts = integrate_rk45_dp('../../src/Comet-Sun.npy',dt,steps,thresh=1e-12)
xs,ys = Rs[:,:2,0].T
xp,yp = Rs[:,:2,1].T
times = np.linspace(0,steps*dt,steps)

## Generate Output 

In [4]:
def makeplot(i):
    xs,ys = Rs[:i,:2,0].T
    xp,yp = Rs[:i,:2,1].T
    relative_error = Es[:i]
    times = np.linspace(0,steps*dt,steps)


    fig,ax = plt.subplots(ncols=3,figsize=(30,10))
    ax[0].scatter(xs[-1],ys[-1],c='k',marker='*',s=100)
    ax[0].plot(xp,yp,c='k',alpha=0.3)
    ax[0].scatter(xp[-1],yp[-1],c='k',s=100)
    ax[0].set_xlabel("X [AU]")
    ax[0].set_ylabel("Y [AU]")
    bound = np.max(np.sqrt(xp**2 + yp**2))
    ax[0].set_xlim(-1.25*np.max(bound),1.25*np.max(bound))
    ax[0].set_ylim(-1.25*np.max(bound),1.25*np.max(bound))

    ax[1].scatter(times[:i],Dts[:i],c='k',alpha=0.3)
    ax[1].set_ylabel(r"$\Delta t$")
    ax[1].set_xlabel("Time [yrs]")
    ax[1].set_ylim(0,1.25*np.max(Dts))
    ax[1].set_xlim(0,np.max(times))

    ax[2].plot(times[:i],relative_error,c='k')
    ax[2].set_ylabel("Energy(t)")
    ax[2].set_xlabel("Time [yrs]")
    ax[2].set_xlim(0,np.max(times))
    ax[2].set_ylim(np.min(Es),np.max(Es))
    return fig

In [6]:
parameter_grid = list(range(1,len(Rs),50)) #plot every 5th timestep
savefigures(makeplot,parameter_grid,outdir='./rk45_dp/')
render('./rk45_dp/','rk45_dp','gif',cleanup_type='rm')

saving figures


100%|█████████████████████████████████████████| 400/400 [06:13<00:00,  1.07it/s]
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora 

Framerate is None fps
Total Runtime is None s


[Parsed_palettegen_0 @ 0x558dfeb73e80] 255(+1) colors generated out of 256 colors; ratio=0.996094
Output #0, image2, to '../temp_palette_rk45_dp.png':
  Metadata:
    encoder         : Lavf58.76.100
  Stream #0:0: Video: png, rgba(pc, gbr/unknown/unknown, progressive), 16x16 [SAR 1:1 DAR 1:1], q=2-31, 200 kb/s, 25 fps, 25 tbn
    Metadata:
      encoder         : Lavc58.134.100 png
frame=    1 fps=0.0 q=-0.0 Lsize=N/A time=00:00:00.04 bitrate=N/A speed=0.00162x    
video:1kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: unknown
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b 